In [1]:
import json
from itertools import chain
from pathlib import Path
from functools import lru_cache
from urllib.request import urlopen

import pandas as pd
import pymorphy2
import regex as re

# typing

from collections.abc import Iterator
from typing import Callable

from pymorphy2.analyzer import Parse

### Вспомогательные объекты

In [2]:
out_dir = Path("./out")
out_dir.mkdir(exist_ok=True)

In [3]:
def task_reader(
    file: Path | str = Path("./data/text.txt"),
) -> Iterator[tuple[str, list[str]]]:
    file = Path(file)
    text = file.read_text()
    for part in text.split("), "):
        context, words = part.split("(", 1)
        context = context.strip()
        words = words.split(", ")
        yield context, words

list(task_reader())

[('Купить килограмм',
  ['абрикос', 'ананас', 'апельсин', 'мандарин', 'баклажан', 'помидор']),
 ('пачка', ['вафля']),
 ('поджарить шесть', ['гренок', 'гренка', 'оладья']),
 ('много вкусных', ['кушанье']),
 ('семь', ['бадья', 'грабли']),
 ('несколько', ['клеймо']),
 ('пять', ['коромысло', 'кочерга', 'шило']),
 ('нет', ['зеркало', 'зеркальце', 'одеяло', 'одеяльце', 'полотенце']),
 ('пара', ['боты', 'бутсы', 'кеды']),
 ('нет', ['гамаши', 'гетры', 'джинсы']),
 ('пара', ['носки', 'сапоги', 'чулки']),
 ('китель без', ['погоны']),
 ('пара', ['клипсы']),
 ('вынуть из', ['ножны']),
 ('десять', ['ватт', 'вольт', 'киловатт']),
 ('засеять пять', ['гектар']),
 ('двести', ['грамм', 'килограмм', 'километр']),
 ('бриллиант в несколько', ['карат']),
 ('идти с дальних', ['верховье', 'взгорье']),
 ('подарить букет', ['георгин']),
 ('несколько', ['русло']),
 ('ряд', ['скирда']),
 ('ждать', ['сумерки']),
 ('из новых', ['ясли']),
 ('много', ['гончарня', 'домна', 'каменоломня', 'прядильня', 'псарня']),
 ('мн

## Задание №1
Загрузить файл text.txt c заданиями, руками привести существительные в скобках в форму родительного падежа. <br>
Сохранить результаты в файл задание1.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}

In [4]:
result_lines: list[str] = []

for context, words in task_reader():
    for word in words:
        target = input(f"{context} ({word}): ")
        line = f"{context} - {target}"
        result_lines.append(line)

(out_dir / "задание1.txt").write_text("\n".join(result_lines))

1345

## Задание №2
Воспользоваться уже сформированным list_task. Воспользуйтесь словарём сформированным из Викисловаря повторите задание №1 <br>
Для слов которых нет в словаре, попробуйте создать сами конструкцию из if-ов которые переведут слово в родительный падаж.<br>
Для более точного подхода можно воспользоваться https://ru.wiktionary.org/wiki/Викисловарь:Шаблоны_словоизменений/Существительные <br>

Сохраните результаты в файл задание2.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}

### Подготовка словаря

In [5]:
df = pd.read_csv('./data/data_lem.csv', index_col=0)
df

,Лемма,Тип речи,Падежы\Модификации
0,Антон,Q147276,"[['Антон', ['Q110786', 'Q131105']], ['Антона',..."
1,всё,Q380057,"[['всё', []]]"
2,ку-ку,Q170239,"[['ку-ку', []]]"
3,метель,Q1084,"[['метель', ['Q110786', 'Q131105']], ['метели'..."
4,август,Q1084,"[['август', ['Q110786', 'Q131105']], ['августа..."
...,...,...,...
101929,кварцевый,Q34698,"[['кварцевый', ['Q110786', 'Q131105', 'Q499327..."
101930,анархо-синдикализм,Q1084,"[['анархо-синдикализм', ['Q110786', 'Q131105']..."
101931,аханье,Q1084,"[['аханье', ['Q110786', 'Q131105']], ['аханья'..."
101932,яванский язык,Q184511,[]


In [6]:
@lru_cache  # cache responses to reduce number of requests
def get_entity_label(entity_code: str) -> str:
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_code}.json"
    content = json.load(urlopen(url))
    try:
        return content["entities"][entity_code]["labels"]["ru"]["value"]
    except Exception:
        print(content)
        raise

get_entity_label("Q147276")

'имя собственное'

In [7]:
df['entity_label'] = df['Тип речи'].map(get_entity_label)
df

,Лемма,Тип речи,Падежы\Модификации,entity_label
0,Антон,Q147276,"[['Антон', ['Q110786', 'Q131105']], ['Антона',...",имя собственное
1,всё,Q380057,"[['всё', []]]",наречие
2,ку-ку,Q170239,"[['ку-ку', []]]",звукоподражательное слово
3,метель,Q1084,"[['метель', ['Q110786', 'Q131105']], ['метели'...",имя существительное
4,август,Q1084,"[['август', ['Q110786', 'Q131105']], ['августа...",имя существительное
...,...,...,...,...
101929,кварцевый,Q34698,"[['кварцевый', ['Q110786', 'Q131105', 'Q499327...",прилагательное
101930,анархо-синдикализм,Q1084,"[['анархо-синдикализм', ['Q110786', 'Q131105']...",имя существительное
101931,аханье,Q1084,"[['аханье', ['Q110786', 'Q131105']], ['аханья'...",имя существительное
101932,яванский язык,Q184511,[],идиома


In [8]:
entity_labels = df["entity_label"].unique()
print('\n'.join(f"{i: >2}: {x}" for i, x in enumerate(entity_labels)))

 0: имя собственное
 1: наречие
 2: звукоподражательное слово
 3: имя существительное
 4: глагол
 5: имя числительное
 6: причастие
 7: прилагательное
 8: предлог
 9: фразеологизм
10: приставка
11: частица
12: именная группа
13: идиома
14: междометие
15: аббревиатура
16: союз
17: деепричастие
18: словоформа
19: личное местоимение
20: символ
21: местоимение
22: суффикс
23: указательное местоимение
24: местоимение-существительное
25: предикатив
26: пословица
27: интерфикс
28: идиоматическое выражение
29: вводное слово
30: суффикс прилагательного
31: местоименное наречие


In [9]:
noun_labels = entity_labels[[0, 3]]
noun_labels

array(['имя собственное', 'имя существительное'], dtype=object)

In [10]:
nouns_df = df[df['entity_label'].isin(noun_labels)].reset_index(drop=True)
nouns_df

,Лемма,Тип речи,Падежы\Модификации,entity_label
0,Антон,Q147276,"[['Антон', ['Q110786', 'Q131105']], ['Антона',...",имя собственное
1,метель,Q1084,"[['метель', ['Q110786', 'Q131105']], ['метели'...",имя существительное
2,август,Q1084,"[['август', ['Q110786', 'Q131105']], ['августа...",имя существительное
3,тыква,Q1084,"[['тыква', ['Q110786', 'Q131105']], ['тыквы', ...",имя существительное
4,кот,Q1084,"[['кот', ['Q110786', 'Q131105']], ['кота', ['Q...",имя существительное
...,...,...,...,...
101282,нос,Q1084,"[['нос', ['Q110786', 'Q131105']], ['носа', ['Q...",имя существительное
101283,дойра,Q1084,"[['дойра', ['Q110786', 'Q131105']], ['дойры', ...",имя существительное
101284,Пермь,Q147276,"[['Пермь', ['Q110786', 'Q131105']], ['Перми', ...",имя собственное
101285,анархо-синдикализм,Q1084,"[['анархо-синдикализм', ['Q110786', 'Q131105']...",имя существительное


In [11]:
def get_form_name(labels: Iterator[str]) -> str:
    return "+".join(sorted(labels))

def parse_forms(raw: str) -> dict[str, str]:
    try:
        content: list[tuple[str, list[str]]] = json.loads(raw.replace("'", '"'))
    except Exception:
        print(raw)
        raise
    result = {
        get_form_name(get_entity_label(entity_code) for entity_code in entities): form
        for form, entities in content
    }
    return result

In [12]:
nouns_df['forms'] = nouns_df["Падежы\\Модификации"].map(parse_forms)
nouns_df['forms']

0         {'единственное число+именительный падеж': 'Ант...
1         {'единственное число+именительный падеж': 'мет...
2         {'единственное число+именительный падеж': 'авг...
3         {'единственное число+именительный падеж': 'тык...
4         {'единственное число+именительный падеж': 'кот...
                                ...                        
101282    {'единственное число+именительный падеж': 'нос...
101283    {'единственное число+именительный падеж': 'дой...
101284    {'единственное число+именительный падеж': 'Пер...
101285    {'единственное число+именительный падеж': 'ана...
101286    {'единственное число+именительный падеж': 'аха...
Name: forms, Length: 101287, dtype: object

In [13]:
get_entity_label.cache_info()  # hits is the number of requests lru_cache avoided

CacheInfo(hits=2580697, misses=52, maxsize=128, currsize=52)

In [14]:
all_form_names = list(nouns_df['forms'].iloc[0].keys())
all_form_names

['единственное число+именительный падеж',
 'единственное число+родительный падеж',
 'дательный падеж+единственное число',
 'единственное число+творительный падеж',
 'единственное число+предложный падеж',
 'винительный падеж+единственное число',
 'именительный падеж+множественное число',
 'множественное число+родительный падеж',
 'винительный падеж+множественное число',
 'множественное число+творительный падеж',
 'множественное число+предложный падеж',
 'дательный падеж+множественное число']

In [15]:
def find_forms(series: pd.Series, form_name: str): 
    def find_form(forms: dict[str, str]):
        return forms.get(form_name, pd.NA)
    return series.map(find_form)
    
for form_name in all_form_names:
    nouns_df[form_name] = nouns_df['forms'].pipe(find_forms, form_name)

nouns_df

,Лемма,Тип речи,Падежы\Модификации,entity_label,forms,единственное число+именительный падеж,единственное число+родительный падеж,дательный падеж+единственное число,единственное число+творительный падеж,единственное число+предложный падеж,винительный падеж+единственное число,именительный падеж+множественное число,множественное число+родительный падеж,винительный падеж+множественное число,множественное число+творительный падеж,множественное число+предложный падеж,дательный падеж+множественное число
0,Антон,Q147276,"[['Антон', ['Q110786', 'Q131105']], ['Антона',...",имя собственное,{'единственное число+именительный падеж': 'Ант...,Антон,Антона,Антону,Антоном,Антоне,Антона,Антоны,Антонов,Антонов,Антонами,Антонах,Антонам
1,метель,Q1084,"[['метель', ['Q110786', 'Q131105']], ['метели'...",имя существительное,{'единственное число+именительный падеж': 'мет...,метель,метели,метели,метелью,метели,метель,метели,метелей,метели,метелями,метелях,метелям
2,август,Q1084,"[['август', ['Q110786', 'Q131105']], ['августа...",имя существительное,{'единственное число+именительный падеж': 'авг...,август,августа,августу,августом,августе,август,августы,августов,августы,августами,августах,августам
3,тыква,Q1084,"[['тыква', ['Q110786', 'Q131105']], ['тыквы', ...",имя существительное,{'единственное число+именительный падеж': 'тык...,тыква,тыквы,тыкве,тыквой,тыкве,тыкву,тыквы,тыкв,тыквы,тыквами,тыквах,тыквам
4,кот,Q1084,"[['кот', ['Q110786', 'Q131105']], ['кота', ['Q...",имя существительное,{'единственное число+именительный падеж': 'кот...,кот,кота,коту,котом,коте,кота,коты,котов,котов,котами,котах,котам
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101282,нос,Q1084,"[['нос', ['Q110786', 'Q131105']], ['носа', ['Q...",имя существительное,{'единственное число+именительный падеж': 'нос...,нос,носа,носу,носом,носе,нос,носы,носов,носы,носами,носах,носам
101283,дойра,Q1084,"[['дойра', ['Q110786', 'Q131105']], ['дойры', ...",имя существительное,{'единственное число+именительный падеж': 'дой...,дойра,дойры,дойре,дойрой,дойре,дойру,дойры,дойр,дойры,дойрами,дойрах,дойрам
101284,Пермь,Q147276,"[['Пермь', ['Q110786', 'Q131105']], ['Перми', ...",имя собственное,{'единственное число+именительный падеж': 'Пер...,Пермь,Перми,Перми,Пермью,Перми,Пермь,Перми,Пермей,Перми,Пeрмями,Пeрмях,Пeрмям
101285,анархо-синдикализм,Q1084,"[['анархо-синдикализм', ['Q110786', 'Q131105']...",имя существительное,{'единственное число+именительный падеж': 'ана...,анархо-синдикализм,анархо-синдикализма,анархо-синдикализму,анархо-синдикализмом,анархо-синдикализме,анархо-синдикализм,анархо-синдикализмы,анархо-синдикализмов,анархо-синдикализмы,анархо-синдикализмами,анархо-синдикализмах,анархо-синдикализмам


In [16]:
forms_df = (
    nouns_df.drop(columns=["Тип речи", "Падежы\\Модификации", "entity_label", "forms"])
    .drop_duplicates(["Лемма"])
    .set_index("Лемма", inplace=False)
)
forms_df

,единственное число+именительный падеж,единственное число+родительный падеж,дательный падеж+единственное число,единственное число+творительный падеж,единственное число+предложный падеж,винительный падеж+единственное число,именительный падеж+множественное число,множественное число+родительный падеж,винительный падеж+множественное число,множественное число+творительный падеж,множественное число+предложный падеж,дательный падеж+множественное число
Лемма,,,,,,,,,,,,
Антон,Антон,Антона,Антону,Антоном,Антоне,Антона,Антоны,Антонов,Антонов,Антонами,Антонах,Антонам
метель,метель,метели,метели,метелью,метели,метель,метели,метелей,метели,метелями,метелях,метелям
август,август,августа,августу,августом,августе,август,августы,августов,августы,августами,августах,августам
тыква,тыква,тыквы,тыкве,тыквой,тыкве,тыкву,тыквы,тыкв,тыквы,тыквами,тыквах,тыквам
кот,кот,кота,коту,котом,коте,кота,коты,котов,котов,котами,котах,котам
...,...,...,...,...,...,...,...,...,...,...,...,...
нос,нос,носа,носу,носом,носе,нос,носы,носов,носы,носами,носах,носам
дойра,дойра,дойры,дойре,дойрой,дойре,дойру,дойры,дойр,дойры,дойрами,дойрах,дойрам
Пермь,Пермь,Перми,Перми,Пермью,Перми,Пермь,Перми,Пермей,Перми,Пeрмями,Пeрмях,Пeрмям


In [17]:
forms_df['множественное число+родительный падеж'].loc['Антон']

'Антонов'

### Решение

In [18]:
result_lines: list[str] = []
form_name = 'множественное число+родительный падеж'

# save indices of context and word to restore order later
hits: list[tuple[int, int, str]] = [] 
misses: list[tuple[int, int, str]] = []

for i, (context, words) in enumerate(task_reader()):
    for j, word in enumerate(words):
        try:
            hit = forms_df[form_name].at[word]
            hits.append((i, j, hit))
        except KeyError:
            misses.append((i, j, word))

print(f"success rate: {len(misses) / (len(hits) + len(misses)) * 100:.2f}%")
print(f"hits:         {', '.join(w for _, _, w in hits)}")
print(f"misses:       {', '.join(w for _, _, w in misses)}")

success rate: 43.84%
hits:         абрикосов, ананасов, апельсинов, мандаринов, баклажанов, помидоров, вафель, гренков, гренок, оладий, бадей, грабель, клейм, коромысел, кочерёг, зеркал, зеркалец, одеял, полотенец, джинсов, гектаров, грамм, килограмм, километров, георгинов, русел, скирд, яслей, гончарен, домен, каменоломен, прядилен, псарен, сабель, армян, калмыков, таджиков, узбеков, якутов, клешней, розог
misses:       кушанье, шило, одеяльце, боты, бутсы, кеды, гамаши, гетры, носки, сапоги, чулки, погоны, клипсы, ножны, ватт, вольт, киловатт, карат, верховье, взгорье, сумерки, копье, гланды, грузин, татарин, дядя, зять, гренадер, драгун, улан, сапер, щупальце


In [19]:
# choose a substitution function based on regexp
substitutions: dict[re.Pattern[str], Callable[[re.Match[str]], str]] = {
    re.compile(r"лки$"): lambda _: "лок",
    re.compile(r"рки$"): lambda _: "рек",
    re.compile(r"ши$"): lambda _: "шей",
    re.compile(r"[ыи]$"): lambda _: "ов",
    re.compile(r"льце$"): lambda _: "лец",
    re.compile(r"ье$"): lambda _: "ий",
    re.compile(r"[цкнгшщзхфвпрлджчмтб]$"): lambda m: m[0] + "ов",
    re.compile(r"[о]$"): lambda _: "",
    re.compile(r"[яь]$"): lambda _: "ей",
}

corrected: list[tuple[int, int, str]] = []

for i, j, word in misses:
    for pattern, repl in substitutions.items():
        if match := pattern.search(word):
            target = pattern.subf(repl, word)
            corrected.append((i, j, target))
            print(f"{word} -> {target}")
            break
    else:
        print(word)

кушанье -> кушаний
шило -> шил
одеяльце -> одеялец
боты -> ботов
бутсы -> бутсов
кеды -> кедов
гамаши -> гамашей
гетры -> гетров
носки -> носков
сапоги -> сапогов
чулки -> чулок
погоны -> погонов
клипсы -> клипсов
ножны -> ножнов
ватт -> ваттов
вольт -> вольтов
киловатт -> киловаттов
карат -> каратов
верховье -> верховий
взгорье -> взгорий
сумерки -> сумерек
копье -> копий
гланды -> гландов
грузин -> грузинов
татарин -> татаринов
дядя -> дядей
зять -> зятей
гренадер -> гренадеров
драгун -> драгунов
улан -> уланов
сапер -> саперов
щупальце -> щупалец


In [20]:
contexts = [c for c, _ in task_reader()]
result_lines: list[str] = []

for i, _, target in sorted(chain(hits, corrected)):
    context = contexts[i]
    line = f"{context} - {target}"
    result_lines.append(line)

(out_dir / "задание2.txt").write_text('\n'.join(result_lines))

1371

## Задание №3
Воспользоваться уже сформированным list_task. Воспользуйтесь библиотекой pymorphy2 и повторите задание №1 <br>
Для слов которых нет в словаре создайте конструкцию из if-ов которые переведут слово в родительный падеж <br>
Сохраните результаты в файл задание3.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}

In [21]:
morph = pymorphy2.MorphAnalyzer()
result_lines: list[str] = []

for context, words in task_reader():
    for word in words:
        parsed: list[Parse] = morph.parse(word)  # type: ignore
        inflected_word: str | None = None
        for p in parsed:  # pick first valid parse
            inflected = p.inflect({"gent", "plur"})
            if isinstance(inflected, Parse) and inflected.word:
                inflected_word = inflected.word
        if not inflected_word:
            raise ValueError(f"could not inflect {parsed}")
        line = f"{context} - {inflected_word}"
        result_lines.append(line)

(out_dir / "задание3.txt").write_text("\n".join(result_lines))

1340